<a href="https://colab.research.google.com/github/purnimapatel/Comparing-of-Deep_learning-Neural-Network/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#import the libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers.core import Dense, Dropout


In [3]:
#opening the zip-file
from zipfile import ZipFile
file_name="UCI HAR Dataset.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print("DONE")

DONE


In [4]:
#those are separate normalised input features for the neural network
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]
# Output classes to learn how to classify
LABELS = [
    "WALKING",
    "WALKING_UPSTAIRS",
    "WALKING_DOWNSTAIRS",
    "SITTING",
    "STANDING",
    "LAYING"
]

In [5]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load_signals
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'/content/UCI HAR Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        #/content/UCI HAR Dataset/train/Inertial Signals
        #/content/UCI HAR Dataset/train/Inertial Signals/body_acc_x_train.txt
        signals_data.append(
            _read_csv(filename).to_numpy()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [6]:
def load_y(subset):
    filename = f'/content/UCI HAR Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).to_numpy()

In [7]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, Y_train, Y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    Y_train, Y_test = load_y('train'), load_y('test')

    return X_train, Y_train, X_test,  Y_test

In [8]:
# Loading the train and test data
X_train, Y_train, X_test,  Y_test = load_data()

In [9]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(7352, 128, 9)
(7352, 6)
(2947, 128, 9)
(2947, 6)


In [10]:
from sklearn.base import BaseEstimator, TransformerMixin
class scaling_tseries_data(BaseEstimator, TransformerMixin):
    from sklearn.preprocessing import StandardScaler
    def __init__(self):
        self.scale = None

    def transform(self, X):
        temp_X1 = X.reshape((X.shape[0] * X.shape[1], X.shape[2]))
        temp_X1 = self.scale.transform(temp_X1)
        return temp_X1.reshape(X.shape)

    def fit(self, X):
        # remove overlaping
        remove = int(X.shape[1] / 2)
        temp_X = X[:, -remove:, :]
        # flatten data
        temp_X = temp_X.reshape((temp_X.shape[0] * temp_X.shape[1], temp_X.shape[2]))
        scale = StandardScaler()
        scale.fit(temp_X)
        self.scale = scale
        return self

In [11]:
Scale = scaling_tseries_data()
Scale.fit(X_train)
X_train_sc = Scale.transform(X_train)
X_test_sc = Scale.transform(X_test)

In [12]:
print('Scaled_X_train',X_train_sc.shape)
print('Scaled_X_test',X_test_sc.shape)

Scaled_X_train (7352, 128, 9)
Scaled_X_test (2947, 128, 9)


**Base_Model of CNN**

In [13]:
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu',kernel_initializer='he_uniform',input_shape=(128,9)))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu',kernel_initializer='he_uniform'))
model.add(Dropout(0.6))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(6, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 126, 32)           896       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 124, 32)           3104      
_________________________________________________________________
dropout (Dropout)            (None, 124, 32)           0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 62, 32)            0         
_________________________________________________________________
flatten (Flatten)            (None, 1984)              0         
_________________________________________________________________
dense (Dense)                (None, 50)                99250     
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 3

In [14]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
model.fit(X_train_sc,Y_train, epochs=45, batch_size=16,validation_data=(X_test_sc, Y_test), verbose=1)

Epoch 1/45
460/460 [==============================] - 4s 8ms/step - loss: 0.4015 - accuracy: 0.8504 - val_loss: 0.2982 - val_accuracy: 0.8958
Epoch 2/45
460/460 [==============================] - 4s 8ms/step - loss: 0.1246 - accuracy: 0.9464 - val_loss: 0.3314 - val_accuracy: 0.8941
Epoch 3/45
460/460 [==============================] - 4s 8ms/step - loss: 0.1039 - accuracy: 0.9578 - val_loss: 0.2686 - val_accuracy: 0.9080
Epoch 4/45
460/460 [==============================] - 4s 8ms/step - loss: 0.1044 - accuracy: 0.9546 - val_loss: 0.3231 - val_accuracy: 0.9074
Epoch 5/45
460/460 [==============================] - 4s 8ms/step - loss: 0.0906 - accuracy: 0.9574 - val_loss: 0.3403 - val_accuracy: 0.8897
Epoch 6/45
460/460 [==============================] - 4s 8ms/step - loss: 0.0881 - accuracy: 0.9596 - val_loss: 0.3297 - val_accuracy: 0.9040
Epoch 7/45
460/460 [==============================] - 4s 8ms/step - loss: 0.0766 - accuracy: 0.9652 - val_loss: 0.3619 - val_accuracy: 0.9087
Epoch 

In [16]:
Y_pred = model.predict(X_test_sc)

In [17]:
pred = np.argmax(Y_pred,axis = 1) 
Y_actual = np.argmax(Y_test,axis = 1)

In [18]:
confusion_matrix(Y_actual, pred)

array([[464,   3,  27,   1,   1,   0],
       [  6, 436,  29,   0,   0,   0],
       [  1,   0, 419,   0,   0,   0],
       [  3,   5,   0, 385,  77,  21],
       [  1,   1,   0, 115, 415,   0],
       [  0,   0,   0,   0,   0, 537]])

In [19]:
print(classification_report(Y_actual, pred))

              precision    recall  f1-score   support

           0       0.98      0.94      0.96       496
           1       0.98      0.93      0.95       471
           2       0.88      1.00      0.94       420
           3       0.77      0.78      0.78       491
           4       0.84      0.78      0.81       532
           5       0.96      1.00      0.98       537

    accuracy                           0.90      2947
   macro avg       0.90      0.90      0.90      2947
weighted avg       0.90      0.90      0.90      2947

